# Lab 3 - Entropy of a string and Huffman coding
## Authors:
|Name|Surname|Student ID|E-mail|
|---|---|---|---|
|Kamal|Ahmadov|22022692|kamal.ahmadov1@ufaz.az|   
|Murad|Mustafayev|22022733 |murad.mustafayev@ufaz.az|

## 1. Entropy of a character string
Recall the formula presented during the lecture for computing the entropy of a character string:
$$H(f) = - \sum_{v=1}^{n} p(v) \log_2(p(v))$$
where $v$ is a byte of the string and $p(v)$ is its probability of being in the string.

## 1.1 Implement a method to compute and store the occurences of each character in a string


In [4]:
import math
import heapq
from collections import Counter, defaultdict

In [5]:
def compute_occurrences(string):
    """
    Compute the occurrences of each character in the given string.

    Args:
        string (str): The input string.

    Returns:
        dict: A dictionary where keys are characters and values are their occurrences.
        int: Total number of characters in the string.
    """
    occurrences = {}
    total_chars = len(string)

    # Iterate over each character in the string
    for char in string:
        # Update the occurrences dictionary
        if char in occurrences:
            occurrences[char] += 1
        else:
            occurrences[char] = 1

    return occurrences, total_chars

## 1.2  Implement a method to compute the entropy of a character string.

In [6]:
def compute_entropy(string):
    """
    Compute the entropy of the given string.

    Args:
        string (str): The input string.

    Returns:
        float: The entropy value of the string.
    """
    # Compute the occurrences of each character and the total number of characters
    occurrences, total_chars = compute_occurrences(string)
    entropy = 0.0

    # Calculate the entropy using the formula H(f) = - \sum_{v=1}^{n} p(v) \log_2(p(v))
    for char, count in occurrences.items():
        probability = count / total_chars
        entropy -= probability * math.log2(probability)

    return entropy

In [8]:
# Test the functions
test_string = "hello world"
entropy = compute_entropy(test_string)
print("Entropy of '{}' is: {:.4f}".format(test_string, entropy))

Entropy of 'hello world' is: 2.8454
Entropy of 'she sells seashells by the seashore' is: 3.0361
Entropy of 'peter piper picked a peck of pickled peppers' is: 3.3412


## 2.  Huffman coding

## 2.1 Design a class `Node` that represents a node in a binary tree: a node is characterized by a value, a left child and a right child. If the node is a leaf, then both children are `null`.

In [9]:
class Node:
    def __init__(self, character, frequency):
        self.character = character
        self.frequency = frequency
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequency < other.frequency


## 2.2. Implement a method to compute the optimal tree to encode given character string.

In [10]:
def build_huffman_tree(text):
    frequency_map = defaultdict(int)
    for c in text:
        frequency_map[c] += 1

    priority_queue = []
    for char, freq in frequency_map.items():
        heapq.heappush(priority_queue, Node(char, freq))

    while len(priority_queue) > 1:
        left = heapq.heappop(priority_queue)
        right = heapq.heappop(priority_queue)

        parent = Node('\0', left.frequency + right.frequency)
        parent.left = left
        parent.right = right
        heapq.heappush(priority_queue, parent)

    return heapq.heappop(priority_queue)

def compute_average_bits(root):
    return compute_average_bits_helper(root, 0, 0) / root.frequency

def compute_average_bits_helper(node, depth, path_length):
    if node is None:
        return 0
    if node.left is None and node.right is None:
        return depth * node.frequency
    return compute_average_bits_helper(node.left, depth + 1, path_length + 1) + \
           compute_average_bits_helper(node.right, depth + 1, path_length + 1)

def build_encoding_map(node, encoding, encoding_map):
    if node is None:
        return
    if node.left is None and node.right is None:
        encoding_map[node.character] = encoding
        return
    build_encoding_map(node.left, encoding + '0', encoding_map)
    build_encoding_map(node.right, encoding + '1', encoding_map)

def encode_string(text, root):
    encoding_map = {}
    build_encoding_map(root, "", encoding_map)

    encoded_string = ""
    for c in text:
        encoded_string += encoding_map[c]
    return encoded_string

def decode_string(encoded_string, root):
    decoded_string = ""
    current = root
    for bit in encoded_string:
        if bit == '0':
            current = current.left
        elif bit == '1':
            current = current.right
        if current.left is None and current.right is None:
            decoded_string += current.character
            current = root
    return decoded_string

## 2.2.(a) Test your method with the following sentences: “peter piper picked a peck of pickled peppers” and “she sells seashells by the seashore”


In [17]:
sentence1 = "peter piper picked a peck of pickled peppers"
tree1 = build_huffman_tree(sentence1)
average_bits1 = compute_average_bits(tree1)
encoded1 = encode_string(sentence1, tree1)
decoded1 = decode_string(encoded1, tree1)
entropy1 = compute_entropy(sentence1)

print("\nSentence 1:")
print("Average Bits per Character: {:.4f}".format(average_bits1))
print("Entropy: {:.4f}".format(entropy1))
print("Encoded String: {}".format(encoded1))
print("Decoded String: {}".format(decoded1))

sentence2 = "she sells seashells by the seashore"
tree2 = build_huffman_tree(sentence2)
average_bits2 = compute_average_bits(tree2)
encoded2 = encode_string(sentence2, tree2)
decoded2 = decode_string(encoded2, tree2)
entropy2 = compute_entropy(sentence2)

print("\nSentence 2:")
print("Average Bits per Character: {:.4f}".format(average_bits2))
print("Entropy: {:.4f}".format(entropy2))
print("Encoded String: {}".format(encoded2))
print("Decoded String: {}".format(decoded2))


Sentence 1:
Average Bits per Character: 3.3864
Entropy: 3.3412
Encoded String: 01111001001111000110011011011111000110011011101010011110000110000101100111110101001110000110010111001101110101001001111110000110011110101111100000110
Decoded String: peter piper picked a peck of pickled peppers

Sentence 2:
Average Bits per Character: 3.0857
Entropy: 3.0361
Encoded String: 100011111101011101101110110101110001100011110110111011001000010011100000001111110101110001100010101001011111
Decoded String: she sells seashells by the seashore


We notice that for both sentences, the calculated entropy is almost the same as the average number of bits per character.


Huffman coding is a smart way to compress data by giving shorter codes to characters that appear more often. When we calculate the average number of bits used per character, it's very close to the entropy of the string. This similarity shows how Huffman coding effectively reduces the average number of bits needed for encoding.